# make OTU table

Did this with Philippe

In [7]:
import pickle

import biom
import pandas as pd
import numpy as np

In [2]:
MOMS_PI_PATH = '../../data/interim/moms_pi/16s_tables.pkl'

In [3]:
with open(MOMS_PI_PATH, "rb") as f:
    tables = pickle.load(f)

/tmp/ipykernel_1159179/1204112297.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  tables = pickle.load(f)


In [4]:
tables

{'MCVD': 1246 x 162 <class 'biom.table.Table'> with 7185 nonzero entries (3% dense),
 'BC1D': 905 x 47 <class 'biom.table.Table'> with 2038 nonzero entries (4% dense),
 'BRCD': 2374 x 967 <class 'biom.table.Table'> with 31133 nonzero entries (1% dense),
 'BS1D': 2270 x 734 <class 'biom.table.Table'> with 30595 nonzero entries (1% dense),
 'MV1D': 3434 x 2055 <class 'biom.table.Table'> with 121282 nonzero entries (1% dense),
 'MCHD': 956 x 152 <class 'biom.table.Table'> with 3079 nonzero entries (2% dense),
 'BCKD': 2719 x 964 <class 'biom.table.Table'> with 40975 nonzero entries (1% dense),
 'BSTD': 538 x 37 <class 'biom.table.Table'> with 1602 nonzero entries (8% dense),
 'MCKD': 3748 x 2327 <class 'biom.table.Table'> with 286063 nonzero entries (3% dense),
 'MRCD': 6251 x 1725 <class 'biom.table.Table'> with 399902 nonzero entries (3% dense)}

In [5]:
big_otu_table = tables["MCVD"]
big_otu_table.update_ids({col: f"{col}_MCVD" for col in big_otu_table.ids()})

for key in tables:
    if key != "MCVD":
        table = tables[key].copy()
        table.update_ids({col: f"{col}_{key}" for col in table.ids()})
        big_otu_table = big_otu_table.merge(table)

big_otu_table

7665 x 9170 <class 'biom.table.Table'> with 923854 nonzero entries (1% dense)

In [27]:
# Sanity checks #1: number of OTUs >= max over individual tables
assert big_otu_table.shape[0] >= np.max([t.shape[0] for t in tables.values()])

# Sanity check #2: number of samples = sum over individual tables
assert big_otu_table.shape[1] == np.sum([t.shape[1] for t in tables.values()])

In [29]:
big_otu_table_normed = big_otu_table.norm()

In [43]:
assert np.allclose(big_otu_table_normed.sum("sample"), 1)

In [8]:
# Generate a list of site IDs

sites = []
for key in tables:
    sites += [key] * tables[key].shape[1]